#start

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install einops

#feature extract

In [2]:
import torch
import cv2
import numpy as np
import os
import math
import torch.optim as optim
import moviepy.editor as mp
import scipy.io
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
def audiospectogram(pathname):
  clip = mp.VideoFileClip(pathname)
  print(pathname[:-4])
  # Insert Local Audio File Path
  audiopath = "/content/drive/MyDrive/dataset/temp.wav"
  
  clip.audio.write_audiofile(audiopath)
  sr,x = scipy.io.wavfile.read(audiopath)

  ## Parameters: 10ms step, 30ms window
  nstep = int(sr * 0.01)
  nwin  = int(sr * 0.01)
  nfft = nwin

  window = np.hamming(nwin)

  ## will take windows x[n1:n2].  generate
  ## and loop over n2 such that all frames
  ## fit within the waveform
  nn = range(nwin, len(x), nstep)

  X = np.zeros( (len(nn), nfft//2) )

  for i,n in enumerate(nn):
      xseg = x[n-nwin:n]
      z = np.fft.fft(window @ xseg, nfft)
      X[i,:] = np.log(np.abs(z[:nfft//2]))


  ab = cv2.resize(X,(224,X.shape[0]))

  valframe =  ab.shape[0]//224 #number of frames of sound
  dim1 = (valframe)*224
  # print(dim1, val, ab.shape[0])
  Y = (ab[:dim1,0:224].T).reshape(valframe,1,224,224)
  os.remove(audiopath)
  Y = np.asarray(Y)
  return Y
def audioextract(path,start,end):
  # audios = np.asarray(audios)
  audios = []
  labels = []
  count =0  
  for filename in os.listdir(path):
    if count>= start and count <end:
      # break
      specframe = audiospectogram(path + filename)
      audios.append(specframe)
      if filename[5:9] == 'Pain':
        labels.append([1,0])
      else:
        labels.append([0,1])
      
    elif count>=end:
      break
    count+=1
  audio = torch.tensor(np.asarray(audios)).float()
  print(audio.shape)
  del audios
  return audio,labels


In [3]:
FRAMES_INTERVAL = 10
def vidextract(DATA_DIR, start, end):
  preview = []
  videos = []
  audios = []
  labels = []
  # GET MIN VIDEO FRAME PER VIDEO: Videos may have different shapes
  min_video_frames = math.inf 

  for video_file_name in os.listdir(DATA_DIR):
    # emoname = video_file_name[:2]
    # if emoname in emotoval:
    vidcap  = cv2.VideoCapture(DATA_DIR + video_file_name)
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    # print(length)
    if length < min_video_frames:
      min_video_frames = length
  print(min_video_frames)
  count = 0
  # EXTRACT SOME FRAMES FROM VIDEOS
  for video_file_name in os.listdir(DATA_DIR):
    if count>= start and count <end:
      
      print("Loading " + video_file_name + "...")
      # label1 = [0 for i in range(lenemo)]
      print(video_file_name[5:9])
      painname = video_file_name[:2]
      if video_file_name[5:9] == 'Pain':
        labels.append([1,0])
      else:
        labels.append([0,1])


      vidcap  = cv2.VideoCapture(DATA_DIR + video_file_name)
      success, image = vidcap.read()
      considered_frames_counter = 0
      frames = []
      while success:    

        if considered_frames_counter == int(min_video_frames / 10) - 1:
          break

        success,image = vidcap.read()
        if considered_frames_counter == FRAMES_INTERVAL:
          preview.append((video_file_name, cv2.resize(image, (224,224))))
        if success and considered_frames_counter % FRAMES_INTERVAL == 0:
          # print(considered_frames_counter,'hi')
          image = np.transpose(np.asarray(cv2.resize(image, (224,224))), (2, 0, 1))
          frames.append(image)
        
        if success:
          considered_frames_counter += 1

      videos.append(frames)
    elif count >= end:
      break
    count += 1
    # print(count, considered_frames_counter)
              
  # audio = torch.tensor(np.asarray(videos)).float()
  video = torch.tensor(np.asarray(videos)).float() # (batch x frames x channels x height x width)
  print(video.shape)
  del videos
  return video,labels


#Combined arch

In [4]:
 ##  EDITED

class TimeSformervid(nn.Module):
    def __init__(
        self,
        *,
        dim,
        num_frames,
        num_classes,
        image_size = 224,
        patch_size = 16,
        channels = 3,
        depth = 12,
        heads = 8,
        dim_head = 64,
        attn_dropout = 0.,
        ff_dropout = 0.
    ):
        super().__init__()
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_size // patch_size) ** 2
        num_positions = num_frames * num_patches
        patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size
        self.to_patch_embedding = nn.Linear(patch_dim, dim)
        self.pos_emb = nn.Embedding(num_positions + 1, dim)
        self.cls_token = nn.Parameter(torch.randn(1, dim))

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Time attention
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Spatial attention
                PreNorm(dim, FeedForward(dim, dropout = ff_dropout)) # Feed Forward
            ]))

        # self.to_out = nn.Sequential(
        #     nn.LayerNorm(dim),
        #     nn.Linear(dim, num_classes)
        # )

    def forward(self, video):
        b, f, _, h, w, *_, device, p = *video.shape, video.device, self.patch_size

        assert h % p == 0 and w % p == 0, f'height {h} and width {w} of video must be divisible by the patch size {p}'

        n = (h // p) * (w // p)
        video = rearrange(video, 'b f c (h p1) (w p2) -> b (f h w) (p1 p2 c)', p1 = p, p2 = p)
        
        tokens = self.to_patch_embedding(video)

        cls_token = repeat(self.cls_token, 'n d -> b n d', b = b)
        x =  torch.cat((cls_token, tokens), dim = 1)
        x += self.pos_emb(torch.arange(x.shape[1], device = device))

        for (time_attn, spatial_attn, ff) in self.layers:
            x = time_attn(x, 'b (f n) d', '(b n) f d', n = n) + x
            x = spatial_attn(x, 'b (f n) d', '(b f) n d', f = f) + x
            x = ff(x) + x

        cls_token = x[:, 0]
        
        return cls_token#self.to_out(cls_token)
    
class TimeSformeraud(nn.Module):
    def __init__(
        self,
        *,
        dim,
        num_frames,
        num_classes,
        image_size = 224,
        patch_size = 16,
        channels = 1,
        depth = 12,
        heads = 8,
        dim_head = 64,
        attn_dropout = 0.,
        ff_dropout = 0.
    ):
        super().__init__()
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_size // patch_size) ** 2
        num_positions = num_frames * num_patches
        patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size
        self.to_patch_embedding = nn.Linear(patch_dim, dim)
        self.pos_emb = nn.Embedding(num_positions + 1, dim)
        self.cls_token = nn.Parameter(torch.randn(1, dim))

        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Time attention
                PreNorm(dim, Attention(dim, dim_head = dim_head, heads = heads, dropout = attn_dropout)), # Spatial attention
                PreNorm(dim, FeedForward(dim, dropout = ff_dropout)) # Feed Forward
            ]))

        # self.to_out = nn.Sequential(
        #     nn.LayerNorm(dim),
        #     nn.Linear(dim, num_classes)
        # )

    def forward(self, video):
        b, f, _, h, w, *_, device, p = *video.shape, video.device, self.patch_size

        assert h % p == 0 and w % p == 0, f'height {h} and width {w} of video must be divisible by the patch size {p}'

        n = (h // p) * (w // p)
        video = rearrange(video, 'b f c (h p1) (w p2) -> b (f h w) (p1 p2 c)', p1 = p, p2 = p)
        
        tokens = self.to_patch_embedding(video)

        cls_token = repeat(self.cls_token, 'n d -> b n d', b = b)
        x =  torch.cat((cls_token, tokens), dim = 1)
        x += self.pos_emb(torch.arange(x.shape[1], device = device))

        for (time_attn, spatial_attn, ff) in self.layers:
            x = time_attn(x, 'b (f n) d', '(b n) f d', n = n) + x
            x = spatial_attn(x, 'b (f n) d', '(b f) n d', f = f) + x
            x = ff(x) + x

        cls_token = x[:, 0]
        
        return cls_token#self.to_out(cls_token)
class ParentModel(nn.Module):
    def __init__(self, modelA, modelB, modelC):
        super(ParentModel, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.modelC(x)
        return x

#part 1 : 0-3


In [ ]:
PATH = '/content/drive/MyDrive/dataset/mix data/'
start = 0
end = 3
audata , audlabel = audioextract(PATH,start,end)
vidata , vidlabel = vidextract(PATH,start,end)

DIM = 224
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2
NUM_FRAMES = 6
DEPTH = 12
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
ITERATIONS = 20
video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
NUM_FRAMES = 8
audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

comb = nn.Sequential(
            nn.LayerNorm(2*DIM),
            nn.Linear(2*DIM, 2),
            nn.Softmax(dim = 1)
        )
model = ParentModel(video_arc, audio_arc, comb)
# #load older
# PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
# chkpnt = torch.load(PATH)
# model.load_state_dict(chkpnt)

loss_fn = torch.nn.BCELoss()
labels = torch.FloatTensor(audlabel)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

learning_rate = 1e-4
for t in range(15):#ITERATIONS):
  y_pred = model(vidata , audata)

  loss = loss_fn(y_pred, torch.tensor(labels))
  print("#" + str(t), " loss:" + str(loss.item()))
  
  model.zero_grad()
  loss.backward()
  with torch.no_grad():
      for param in model.parameters():
          param -= learning_rate * param.grad
PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
torch.save(model.state_dict(), PATH)

/content/drive/MyDrive/dataset/mix data/S001_Friction_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1431.00it/s]


[MoviePy] Done.
/content/drive/MyDrive/dataset/mix data/S001_Rest_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1637.52it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S001_Rest_2_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1633.72it/s]

[MoviePy] Done.


torch.Size([3, 8, 1, 224, 224])
600
Loading S001_Friction_1_[0]_20s.mp4...
Fric
Loading S001_Rest_1_[0]_20s.mp4...
Rest
Loading S001_Rest_2_[0]_20s.mp4...
Rest
torch.Size([3, 6, 3, 224, 224])
#0  loss:1.2430533170700073
#1  loss:1.1109040975570679
#2  loss:1.1025928258895874
#3  loss:0.8867625594139099
#4  loss:0.8954576849937439
#5  loss:0.8472201824188232
#6  loss:0.7947699427604675
#7  loss:0.7569810748100281
#8  loss:0.6903131604194641
#9  loss:0.612330973148346
#10  loss:0.5015454292297363
#11  loss:0.44943031668663025
#12  loss:0.5118887424468994
#13  loss:0.3870948255062103
#14  loss:0.39312586188316345


#part 2 : 86-90

In [ ]:
PATH = '/content/drive/MyDrive/dataset/mix data/'
start = 86
end = 90
audata , audlabel = audioextract(PATH,start,end)
vidata , vidlabel = vidextract(PATH,start,end)

DIM = 224
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2
NUM_FRAMES = 6
DEPTH = 12
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
ITERATIONS = 20
video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
NUM_FRAMES = 8
audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

comb = nn.Sequential(
            nn.LayerNorm(2*DIM),
            nn.Linear(2*DIM, 2),
            nn.Softmax(dim = 1)
        )
model = ParentModel(video_arc, audio_arc, comb)
# #load older
PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
chkpnt = torch.load(PATH)
model.load_state_dict(chkpnt)

loss_fn = torch.nn.BCELoss()
labels = torch.FloatTensor(audlabel)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

learning_rate = 1e-4
for t in range(15):#ITERATIONS):
  y_pred = model(vidata , audata)

  loss = loss_fn(y_pred, torch.tensor(labels))
  print("#" + str(t), " loss:" + str(loss.item()))
  
  model.zero_grad()
  loss.backward()
  with torch.no_grad():
      for param in model.parameters():
          param -= learning_rate * param.grad
PATH = '/content/drive/MyDrive/dataset/paindata2.pth'
torch.save(model.state_dict(), PATH)

/content/drive/MyDrive/dataset/mix data/S039_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1485.73it/s]


[MoviePy] Done.
/content/drive/MyDrive/dataset/mix data/S038_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1361.27it/s]

[MoviePy] Done.


...

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S042_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1146.38it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S039_Pain_1_[0]_20s.mp4...
Pain
Loading S038_Pain_1_[0]_20s.mp4...
Pain
Loading S040_Pain_1_[0]_20s.mp4...
Pain
Loading S042_Pain_1_[0]_20s.mp4...
Pain
torch.Size([4, 6, 3, 224, 224])
#0  loss:1.077333688735962
#1  loss:1.0384373664855957
#2  loss:0.8980099558830261
#3  loss:0.8334181308746338
#4  loss:0.8182480335235596
#5  loss:0.6857007741928101
#6  loss:0.6618469953536987
#7  loss:0.5908451080322266
#8  loss:0.5738427639007568
#9  loss:0.5837259888648987
#10  loss:0.4521908760070801
#11  loss:0.3856562376022339
#12  loss:0.4569796323776245
#13  loss:0.49451157450675964
#14  loss:0.40760892629623413


#part 3 : 4-8

In [ ]:
PATH = '/content/drive/MyDrive/dataset/mix data/'
start = 4
end = 8
audata , audlabel = audioextract(PATH,start,end)
vidata , vidlabel = vidextract(PATH,start,end)

DIM = 224
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2
NUM_FRAMES = 6
DEPTH = 12
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
ITERATIONS = 20
video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
NUM_FRAMES = 8
audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

comb = nn.Sequential(
            nn.LayerNorm(2*DIM),
            nn.Linear(2*DIM, 2),
            nn.Softmax(dim = 1)
        )
model = ParentModel(video_arc, audio_arc, comb)
# #load older
# PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
# chkpnt = torch.load(PATH)
# model.load_state_dict(chkpnt)

loss_fn = torch.nn.BCELoss()
labels = torch.FloatTensor(audlabel)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

learning_rate = 1e-4
for t in range(15):#ITERATIONS):
  y_pred = model(vidata , audata)

  loss = loss_fn(y_pred, torch.tensor(labels))
  print("#" + str(t), " loss:" + str(loss.item()))
  
  model.zero_grad()
  loss.backward()
  with torch.no_grad():
      for param in model.parameters():
          param -= learning_rate * param.grad
PATH = '/content/drive/MyDrive/dataset/paindata3.pth'
torch.save(model.state_dict(), PATH)

/content/drive/MyDrive/dataset/mix data/S002_Friction_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1125.34it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S002_Rest_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1310.74it/s]

...

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S003_Rest_2_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1424.54it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S002_Friction_1_[0]_20s.mp4...
Fric
Loading S002_Rest_1_[0]_20s.mp4...
Rest
Loading S002_Rest_2_[0]_20s.mp4...
Rest
Loading S003_Rest_2_[0]_20s.mp4...
Rest
torch.Size([4, 6, 3, 224, 224])
#0  loss:0.6782015562057495
#1  loss:0.6673566102981567
#2  loss:0.5652531981468201
#3  loss:0.5255536437034607
#4  loss:0.4472886919975281
#5  loss:0.4757254421710968
#6  loss:0.5204955339431763
#7  loss:0.3610319197177887
#8  loss:0.38808929920196533
#9  loss:0.3485448360443115
#10  loss:0.3161793351173401
#11  loss:0.2966289818286896
#12  loss:0.303041011095047
#13  loss:0.31347766518592834
#14  loss:0.26301833987236023


#part 4 : 82-86

In [ ]:
PATH = '/content/drive/MyDrive/dataset/mix data/'
start = 82
end = 86
audata , audlabel = audioextract(PATH,start,end)
vidata , vidlabel = vidextract(PATH,start,end)

DIM = 224
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2
NUM_FRAMES = 6
DEPTH = 12
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
ITERATIONS = 20
video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
NUM_FRAMES = 8
audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

comb = nn.Sequential(
            nn.LayerNorm(2*DIM),
            nn.Linear(2*DIM, 2),
            nn.Softmax(dim = 1)
        )
model = ParentModel(video_arc, audio_arc, comb)
# #load older
# PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
# chkpnt = torch.load(PATH)
# model.load_state_dict(chkpnt)

loss_fn = torch.nn.BCELoss()
labels = torch.FloatTensor(audlabel)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

learning_rate = 1e-4
for t in range(15):#ITERATIONS):
  y_pred = model(vidata , audata)

  loss = loss_fn(y_pred, torch.tensor(labels))
  print("#" + str(t), " loss:" + str(loss.item()))
  
  model.zero_grad()
  loss.backward()
  with torch.no_grad():
      for param in model.parameters():
          param -= learning_rate * param.grad
PATH = '/content/drive/MyDrive/dataset/paindata4.pth'
torch.save(model.state_dict(), PATH)

/content/drive/MyDrive/dataset/mix data/S035_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1085.72it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S036_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1372.56it/s]

...

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S037_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1339.25it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S035_Pain_1_[0]_20s.mp4...
Pain
Loading S036_Pain_1_[0]_20s.mp4...
Pain
Loading S034_Pain_1_[0]_20s.mp4...
Pain
Loading S037_Pain_1_[0]_20s.mp4...
Pain
torch.Size([4, 6, 3, 224, 224])
#0  loss:1.1664968729019165
#1  loss:1.194850206375122
#2  loss:1.019329309463501
#3  loss:0.9733767509460449
#4  loss:0.8188222646713257
#5  loss:0.8372744917869568
#6  loss:0.7279483079910278
#7  loss:0.666986346244812
#8  loss:0.6948415040969849
#9  loss:0.5190324187278748
#10  loss:0.5269142985343933
#11  loss:0.5003228783607483
#12  loss:0.46582478284835815
#13  loss:0.4574684798717499
#14  loss:0.40359535813331604


#part 5 : 4-8

In [ ]:
PATH = '/content/drive/MyDrive/dataset/mix data/'
start = 8
end = 12
audata , audlabel = audioextract(PATH,start,end)
vidata , vidlabel = vidextract(PATH,start,end)

DIM = 224
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2
NUM_FRAMES = 6
DEPTH = 12
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
ITERATIONS = 20
video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
NUM_FRAMES = 8
audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

comb = nn.Sequential(
            nn.LayerNorm(2*DIM),
            nn.Linear(2*DIM, 2),
            nn.Softmax(dim = 1)
        )
model = ParentModel(video_arc, audio_arc, comb)
# #load older
# PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
# chkpnt = torch.load(PATH)
# model.load_state_dict(chkpnt)

loss_fn = torch.nn.BCELoss()
labels = torch.FloatTensor(audlabel)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

learning_rate = 1e-4
for t in range(15):#ITERATIONS):
  y_pred = model(vidata , audata)

  loss = loss_fn(y_pred, torch.tensor(labels))
  print("#" + str(t), " loss:" + str(loss.item()))
  
  model.zero_grad()
  loss.backward()
  with torch.no_grad():
      for param in model.parameters():
          param -= learning_rate * param.grad
PATH = '/content/drive/MyDrive/dataset/paindata5.pth'
torch.save(model.state_dict(), PATH)

/content/drive/MyDrive/dataset/mix data/S003_Rest_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1436.57it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S003_Friction_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1538.97it/s]

...

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S004_Move_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1251.21it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S003_Rest_1_[0]_20s.mp4...
Rest
Loading S003_Friction_1_[0]_20s.mp4...
Fric
Loading S004_Friction_1_[0]_20s.mp4...
Fric
Loading S004_Move_1_[0]_20s.mp4...
Move
torch.Size([4, 6, 3, 224, 224])
#0  loss:1.1044385433197021
#1  loss:0.9471675157546997
#2  loss:0.8939681053161621
#3  loss:0.8573133945465088
#4  loss:0.7727766633033752
#5  loss:0.6236066818237305
#6  loss:0.6053695678710938
#7  loss:0.5926843285560608
#8  loss:0.5377585291862488
#9  loss:0.5732808709144592
#10  loss:0.43039464950561523
#11  loss:0.4244805872440338
#12  loss:0.401862233877182
#13  loss:0.39070671796798706
#14  loss:0.41018998622894287


#part 6 : 8-12, 74-82

In [ ]:
strl = [78, 8 , 74]
endl = [82, 12, 78 ]
for i in range(3):
  PATH = '/content/drive/MyDrive/dataset/mix data/'
  start = strl[i]
  end = endl[i]
  audata , audlabel = audioextract(PATH,start,end)
  vidata , vidlabel = vidextract(PATH,start,end)

  DIM = 224
  IMAGE_SIZE = 224
  PATCH_SIZE = 16
  NUM_CLASSES = 2
  NUM_FRAMES = 6
  DEPTH = 12
  HEADS = 8
  DIM_HEAD = 64
  ATTN_DROPOUT = 0.1
  FF_DROPOUT = 0.1
  ITERATIONS = 20
  video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
  NUM_FRAMES = 8
  audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

  comb = nn.Sequential(
              nn.LayerNorm(2*DIM),
              nn.Linear(2*DIM, 2),
              nn.Softmax(dim = 1)
          )
  model = ParentModel(video_arc, audio_arc, comb)
  # #load older
  # PATH = '/content/drive/MyDrive/dataset/paindata1.pth'
  # chkpnt = torch.load(PATH)
  # model.load_state_dict(chkpnt)

  loss_fn = torch.nn.BCELoss()
  labels = torch.FloatTensor(audlabel)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

  learning_rate = 1e-4
  for t in range(15):#ITERATIONS):
    y_pred = model(vidata , audata)

    loss = loss_fn(y_pred, torch.tensor(labels))
    print("#" + str(t), " loss:" + str(loss.item()))
    
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
  PATH = '/content/drive/MyDrive/dataset/paindata6.pth'
  torch.save(model.state_dict(), PATH)

/content/drive/MyDrive/dataset/mix data/S031_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1330.94it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S029_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1422.98it/s]

...

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S028_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1355.23it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S026_Pain_1_[0]_20s.mp4...
Pain
Loading S025_Pain_1_[0]_20s.mp4...
Pain
Loading S030_Pain_1_[0]_20s.mp4...
Pain
Loading S028_Pain_1_[0]_20s.mp4...
Pain
torch.Size([4, 6, 3, 224, 224])
#0  loss:0.9065759778022766
#1  loss:0.8021986484527588
#2  loss:0.6452558040618896
#3  loss:0.588513970375061
#4  loss:0.5697724223136902
#5  loss:0.4710915982723236
#6  loss:0.4661558270454407
#7  loss:0.4954458177089691
#8  loss:0.47309213876724243
#9  loss:0.41952943801879883
#10  loss:0.3623654246330261
#11  loss:0.3622218072414398
#12  loss:0.3871691823005676
#13  loss:0.2804570198059082
#14  loss:0.3315197825431824


#PART 7 : 12-20, 64

In [ ]:
# device = 'cuda'
strl = [12, 70 , 16, 66, 20, 62, 24, 58, 28, 54, 32] 50, 36, 46, 40, 74, 82]
endl = [16, 74 , 20, 70, 24, 66, 28, 62, 32, 58, 36] 46, 40, 50, 44, 78, 86]
for i in range(len(strl)):
  print(i)
  PATH = '/content/drive/MyDrive/dataset/mix data/'
  start = strl[i]
  end = endl[i]
  if start - end >4:
    continue
  audata , audlabel = audioextract(PATH,start,end)
  vidata , vidlabel = vidextract(PATH,start,end)
  # audata = audata.to(device)
  # vidata = vidata.to(device)
  if i == 0:
    DIM = 224
    IMAGE_SIZE = 224
    PATCH_SIZE = 16
    NUM_CLASSES = 2
    NUM_FRAMES = 6
    DEPTH = 12
    HEADS = 8
    DIM_HEAD = 64
    ATTN_DROPOUT = 0.1
    FF_DROPOUT = 0.1
    ITERATIONS = 20
    video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
    NUM_FRAMES = 8
    audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

    comb = nn.Sequential(
                nn.LayerNorm(2*DIM),
                nn.Linear(2*DIM, 2),
                nn.Softmax(dim = 1)
            )
    model = ParentModel(video_arc, audio_arc, comb)
    #load older
    
    PATH = '/content/drive/MyDrive/dataset/paindata6.pth'
    chkpnt = torch.load(PATH)
    model.load_state_dict(chkpnt)
    
  # model = model.to(device)

  loss_fn = torch.nn.BCELoss()
  labels = torch.FloatTensor(vidlabel)#.to(device)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

  learning_rate = 1e-4
  for t in range(15):#ITERATIONS):
    y_pred = model(vidata , audata)#.to(device)

    loss = loss_fn(y_pred, torch.tensor(labels))
    print("#" + str(t), " loss:" + str(loss.item()))
    
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
  PATH = '/content/drive/MyDrive/dataset/paindata7.pth'
  torch.save(model.state_dict(), PATH)

0
/content/drive/MyDrive/dataset/mix data/S004_Rest_1_[1]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1166.07it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S004_Rest_2_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1681.29it/s]

...

/content/drive/MyDrive/dataset/mix data/S010_Friction_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1563.28it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S009_Move_1_[0]_20s.mp4...
Move
Loading S009_Rest_1_[1]_20s.mp4...
Rest
Loading S009_Rest_3_[0]_20s.mp4...
Rest
Loading S010_Friction_1_[0]_20s.mp4...
Fric
torch.Size([4, 6, 3, 224, 224])
#0  loss:1.1511452198028564
#1  loss:0.8800007104873657
#2  loss:1.061805009841919
#3  loss:0.9005830883979797
#4  loss:0.7755218148231506
#5  loss:0.7918651700019836
#6  loss:0.6641393899917603
#7  loss:0.6105556488037109
#8  loss:0.6186174750328064
#9  loss:0.5449140667915344
#10  loss:0.4837146997451782
#11  loss:0.48528432846069336
#12  loss:0.45924264192581177
#13  loss:0.46868860721588135
#14  loss:0.3865523934364319
11
torch.Size([0])
600
torch.Size([0])


ValueError: ignored

#part 8: 

In [ ]:
# device = 'cuda'
strl =  [46, 36, 46, 40, 74, 82]
endl = [50, 40, 50, 44, 78, 86]
for i in range(len(strl)):
  
  PATH = '/content/drive/MyDrive/dataset/mix data/'
  start = strl[i]
  end = endl[i]
  print(i,start,end)
  if start - end >4:
    continue
  audata , audlabel = audioextract(PATH,start,end)
  vidata , vidlabel = vidextract(PATH,start,end)
  # audata = audata.to(device)
  # vidata = vidata.to(device)
  if i == 0:
    DIM = 224
    IMAGE_SIZE = 224
    PATCH_SIZE = 16
    NUM_CLASSES = 2
    NUM_FRAMES = 6
    DEPTH = 12
    HEADS = 8
    DIM_HEAD = 64
    ATTN_DROPOUT = 0.1
    FF_DROPOUT = 0.1
    ITERATIONS = 20
    video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
    NUM_FRAMES = 8
    audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

    comb = nn.Sequential(
                nn.LayerNorm(2*DIM),
                nn.Linear(2*DIM, 2),
                nn.Softmax(dim = 1)
            )
    model = ParentModel(video_arc, audio_arc, comb)
    #load older
    
    PATH = '/content/drive/MyDrive/dataset/paindata7.pth'
    chkpnt = torch.load(PATH)
    model.load_state_dict(chkpnt)
    print('done')
    
  # model = model.to(device)

  loss_fn = torch.nn.BCELoss()
  labels = torch.FloatTensor(vidlabel)#.to(device)#[[0.4, 0.6] for i in range(len(video))]) # Add here your own labels

  learning_rate = 1e-4
  for t in range(15):#ITERATIONS):
    y_pred = model(vidata , audata)#.to(device)

    loss = loss_fn(y_pred, torch.tensor(labels))
    print("#" + str(t), " loss:" + str(loss.item()))
    
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
  PATH = '/content/drive/MyDrive/dataset/paindata8.pth'
  torch.save(model.state_dict(), PATH)

0 46 50
/content/drive/MyDrive/dataset/mix data/S013_Friction_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1626.38it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S013_Move_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1600.84it/s]

...

100%|██████████| 442/442 [00:00<00:00, 1491.63it/s]


[MoviePy] Done.
/content/drive/MyDrive/dataset/mix data/S037_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1559.87it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S035_Pain_1_[0]_20s.mp4...
Pain
Loading S036_Pain_1_[0]_20s.mp4...
Pain
Loading S034_Pain_1_[0]_20s.mp4...
Pain
Loading S037_Pain_1_[0]_20s.mp4...
Pain
torch.Size([4, 6, 3, 224, 224])
#0  loss:0.6147356629371643
#1  loss:0.6175612211227417
#2  loss:0.4831807017326355
#3  loss:0.4929247796535492
#4  loss:0.41369664669036865
#5  loss:0.4469064772129059
#6  loss:0.4215494692325592
#7  loss:0.32803112268447876
#8  loss:0.3767380714416504
#9  loss:0.3427108824253082
#10  loss:0.3330773711204529
#11  loss:0.30270856618881226
#12  loss:0.38664478063583374
#13  loss:0.26185429096221924
#14  loss:0.22948701679706573


#eval

In [5]:
DIM = 224
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_CLASSES = 2
NUM_FRAMES = 6
DEPTH = 12
HEADS = 8
DIM_HEAD = 64
ATTN_DROPOUT = 0.1
FF_DROPOUT = 0.1
ITERATIONS = 20
video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
NUM_FRAMES = 8
audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

comb = nn.Sequential(
            nn.LayerNorm(2*DIM),
            nn.Linear(2*DIM, 2),
            nn.Softmax(dim = 1)
        )
model = ParentModel(video_arc, audio_arc, comb)
#load older

PATH = '/content/drive/MyDrive/dataset/paindata8.pth'
chkpnt = torch.load(PATH)
model.load_state_dict(chkpnt)
print('done')

# PATH = '/content/drive/MyDrive/dataset/mix data/'
# start = 59
# end = 62
# audata , audlabel = audioextract(PATH,start,end)
# vidata , vidlabel = vidextract(PATH,start,end)


done


In [6]:
PATH = '/content/drive/MyDrive/dataset/mix data/'
start = 48
end = 52
audata , audlabel = audioextract(PATH,start,end)
vidata , vidlabel = vidextract(PATH,start,end)

y_pred = model(vidata , audata)

/content/drive/MyDrive/dataset/mix data/S013_Rest_2_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1429.99it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S001_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1691.81it/s]

...

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S002_Pain_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1661.67it/s]

[MoviePy] Done.


torch.Size([4, 8, 1, 224, 224])
600
Loading S013_Rest_2_[0]_20s.mp4...
Rest
Loading S001_Pain_1_[0]_20s.mp4...
Pain
Loading S003_Pain_1_[0]_20s.mp4...
Pain
Loading S002_Pain_1_[0]_20s.mp4...
Pain
torch.Size([4, 6, 3, 224, 224])


In [7]:
outGT = np.asarray(vidlabel)
outPRED = np.asarray(torch.Tensor.detach(y_pred))
from sklearn.metrics import *
print('F1: {}'.format(f1_score(outGT, outPRED>.5, average="samples")))
print('Precision: {}'.format(precision_score(outGT, outPRED>.5, average="samples")))
print('Recall: {}'.format(recall_score(outGT, outPRED >.5, average="samples")))
print('Accuracy: {}'.format(accuracy_score(outGT, outPRED>.5)))

F1: 0.75
Precision: 0.75
Recall: 0.75
Accuracy: 0.75


In [ ]:
for i in range(len(y_pred)):
  print(y_pred[i], vidlabel[i] )

tensor([0.8129, 0.1871], grad_fn=<SelectBackward0>) [0, 1]
tensor([0.7894, 0.2106], grad_fn=<SelectBackward0>) [0, 1]
tensor([0.8565, 0.1435], grad_fn=<SelectBackward0>) [0, 1]


In [ ]:
# #Parameters
# DIM = 224
# IMAGE_SIZE = 224
# PATCH_SIZE = 16
# NUM_CLASSES = 2
# NUM_FRAMES = 2
# DEPTH = 12
# HEADS = 8
# DIM_HEAD = 64
# ATTN_DROPOUT = 0.1
# FF_DROPOUT = 0.1
# ITERATIONS = 20
# video_arc = TimeSformervid(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)
# NUM_FRAMES = 8
# audio_arc = TimeSformeraud(dim = DIM, image_size = IMAGE_SIZE, patch_size = PATCH_SIZE, num_frames = NUM_FRAMES, num_classes = NUM_CLASSES, depth = DEPTH, heads = HEADS, dim_head = DIM_HEAD, attn_dropout = ATTN_DROPOUT, ff_dropout = FF_DROPOUT)

# comb = nn.Sequential(
#             nn.LayerNorm(2*DIM),
#             nn.Linear(2*DIM, 6)
#         )
# finmodel = ParentModel(video_arc, audio_arc, comb)

In [ ]:
# PATH = '/content/drive/MyDrive/dataset/mix data/'
# start = 0
# end = 3
# audata , audlabel = audioextract(PATH,start,end)
# vidata , vidlabel = vidextract(PATH,start,end)


/content/drive/MyDrive/dataset/mix data/S001_Friction_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1467.98it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S001_Rest_1_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1505.76it/s]

[MoviePy] Done.


/content/drive/MyDrive/dataset/mix data/S001_Rest_2_[0]_20s
[MoviePy] Writing audio in /content/drive/MyDrive/dataset/temp.wav


100%|██████████| 442/442 [00:00<00:00, 1571.27it/s]

[MoviePy] Done.


torch.Size([3, 8, 1, 224, 224])
600
Loading S001_Friction_1_[0]_20s.mp4...
Fric
Loading S001_Rest_1_[0]_20s.mp4...
Rest
Loading S001_Rest_2_[0]_20s.mp4...
Rest
torch.Size([3, 6, 3, 224, 224])


In [ ]:
# pred = finmodel(vidata, audata)

In [ ]:
pred = model(vidata, audata)

IndexError: ignored